In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import colorcet as cc
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
import sys

sys.path.append("/Users/jlq293/Projects/Study-1-Bibliometrics/src/main_path/")

from MainPathReader import MainPathReader
from MainPathDataAssigner import MainPathDataAssigner

In [3]:
p = "../data/06-clustered-df/alpha0.3_k20_res0.006.pkl"  # cluster_alpha0.3_k20_res0.006
df = pd.read_pickle(p)

p_summary = "../output/tables/cluster-explorer/FinalSelect/SingleSolExplorer_alpha0.3_k20_res0.006.xlsx"
# read sheet 1
df_summary = pd.read_excel(p_summary, sheet_name=0)

Cannot be read by networkx, so reading in from scratch

# read in main path data from pajek


In [4]:
path_to_main_path = (
    "../data/08-main-paths/10. Key-Route Global Main Path [200-250] of N5 (185).net"
)
mp_reader = MainPathReader(
    path_to_main_path, node_attributes=["eid", "unique_auth_year"]
)
Gmp = mp_reader.get_graph()
list(Gmp.nodes(data=True))[:2]

FileNotFoundError: [Errno 2] No such file or directory: '../data/08-main-paths/10. Key-Route Global Main Path [200-250] of N5 (185).net'

# Add Info from df to mp


In [ ]:
attr_to_assign = [
    "title",
    "citedby_count",
    "unique_auth_year",
    "doi",
    "year",
    "cluster_alpha0.3_k20_res0.006",
    "centrality_alpha0.3_k20_res0.006",
]


data_assigner = MainPathDataAssigner(Gmp, df, attr_to_assign)
Gmp = data_assigner.process_mp()
print(list(Gmp.nodes(data=True))[:2])

[('1', {'label': '"family_2_62"', 'eid': '2-s2.0-0020047901;2-s2.0-0019989539', 'unique_auth_year': 'Boeck_1982;Pedersen_1982', 'title': 'Studies on Acute Toxicity and Drug Levels of Citalopram in the Dog;Citalopram, a selective serotonin reuptake inhibitor: Clinical antidepressive and long-term effect - a phase II study', 'citedby_count': '34;61', 'doi': '10.1111/j.1600-0773.1982.tb00959.x;10.1007/BF00464566', 'year': '1982;1982', 'cluster_alpha0.3_k20_res0.006': '47;76', 'centrality_alpha0.3_k20_res0.006': '0.073681300627201;0.3242186570081556'}), ('2', {'label': '"47"', 'eid': '2-s2.0-0020069956', 'unique_auth_year': 'Baker_1982', 'title': 'The effects of the specific uptake inhibitor Lu 10-171 (Citalopram) upon brain indoleamine stores in the maturing mouse', 'citedby_count': '11', 'doi': '10.1016/0306-3623(82)90013-1', 'year': '1982', 'cluster_alpha0.3_k20_res0.006': '67', 'centrality_alpha0.3_k20_res0.006': '0.06512483207962619'})]


# plot the main path


In [ ]:
from MainPathPlotterStatic import MainPathPlotterStatic
from MainPathPlotterInteractive import MainPathPlotterInteractive

In [ ]:
import colorcet as cc
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np


class MainPathPlotterInteractive:
    def __init__(self, G, cluster_col, label_col):
        self.G = G
        self.cluster_col = cluster_col
        self.label_col = label_col

    @staticmethod
    def add_line_breaks(text, char_limit=100):
        words = text.split()
        line = ""
        lines = []
        for word in words:
            if len(line + " " + word) <= char_limit:
                line += " " + word
            else:
                lines.append(line.strip())
                line = word
        if line:
            lines.append(line.strip())
        return "<br>".join(lines)

    def adjust_overlap(self, min_dist=0.1, max_iterations=100):
        """
        Adjusts the positions to reduce overlap.

        :param pos: Dictionary of positions keyed by node.
        :param min_dist: Minimum desired distance between nodes.
        :param max_iterations: Maximum number of iterations to perform adjustments.
        :return: Adjusted positions.
        """
        for _ in range(max_iterations):
            moved = False
            for node1 in self.pos:
                for node2 in self.pos:
                    if node1 != node2:
                        # Calculate the distance between two nodes
                        delta = np.array(self.pos[node1]) - np.array(self.pos[node2])
                        dist = np.sqrt(np.sum(delta**2))
                        # If the nodes are too close, push them apart
                        if dist < min_dist:
                            moved = True
                            displacement = delta / dist * (min_dist - dist) / 2
                            self.pos[node1] += displacement
                            self.pos[node2] -= displacement
            # If no nodes were moved in this iteration, stop the adjustment
            if not moved:
                print(f"Adjustment finished after {_} iterations")
                break
        return self.pos

    def clean_years(self):
        """
        Convert year attribute to integer, if not already
        if multiple years are present, the mean is taken
        """
        # Convert year attribute to integer, if not already
        for node, data in self.G.nodes(data=True):
            if isinstance(data["year"], str):
                mean_year = sum(map(int, data["year"].split(";"))) / len(
                    data["year"].split(";")
                )
                data["year"] = int(mean_year)

        # Sort nodes based on year attribute
        nodes_sorted_by_year = sorted(
            self.G.nodes(data=True), key=lambda x: x[1]["year"], reverse=True
        )

        # Get the range of years
        min_year = min([data["year"] for node, data in nodes_sorted_by_year])
        max_year = max([data["year"] for node, data in nodes_sorted_by_year])

        return min_year, max_year, nodes_sorted_by_year

    def hover_texts(self):
        hover_texts = []
        for node, data in self.G.nodes(data=True):
            wrapped_title = self.add_line_breaks(data["title"])
            hover_title_cluster = (
                f"Cluster: {data[self.cluster_col]}<br>{wrapped_title}"
            )
            hover_title_cluster_cited = f"Cluster: {data[self.cluster_col]}<br>Cited by: {data['citedby_count']}<br>{wrapped_title}"
            hover_texts.append(hover_title_cluster_cited)
        return hover_texts

    def cluster_color_dict(self):
        """
        Creates a dictionary mapping clusters to distinct colors using the colorcet library,
        and returns a list of colors for each node based on its cluster.

        Returns:
            list: A list of color strings in hex format for each node in the graph, based on its cluster.
        """
        clusters = nx.get_node_attributes(self.G, self.cluster_col).values()
        unique_clusters = sorted(set(clusters))

        # Using colorcet to get a colormap with enough distinct colors
        # Adjust the colormap if needed to suit your preferences for color variety and distinctness
        color_list = cc.glasbey_dark[: len(unique_clusters)]

        # Creating a color dictionary mapping each cluster to a color
        color_dict = {
            cluster: color_list[i] for i, cluster in enumerate(unique_clusters)
        }

        # Generating a list of colors for each node based on its cluster
        colors = [color_dict[cluster] for cluster in clusters]
        return colors

    def node_labels(self):
        node_labels = []
        for node, data in self.G.nodes(data=True):
            node_labels.append(data[self.label_col])
        return node_labels

    def edge_positions(self):
        edge_x = []
        edge_y = []
        for edge in self.G.edges():
            x0, y0 = self.pos[edge[0]]
            x1, y1 = self.pos[edge[1]]
            edge_x.append(x0)
            edge_x.append(x1)
            edge_x.append(None)
            edge_y.append(y0)
            edge_y.append(y1)
            edge_y.append(None)
        return edge_x, edge_y

    def get_edge_trace(self):
        edge_x, edge_y = self.edge_positions()
        edge_trace = go.Scatter(
            x=edge_x,
            y=edge_y,
            line=dict(width=0.7, color="#000000"),
            hoverinfo="none",
            mode="lines",
        )
        return edge_trace

    def get_node_trace(self):
        node_x, node_y = zip(*self.pos.values())
        hover_texts = self.hover_texts()
        colors = self.cluster_color_dict()
        node_labels = self.node_labels()
        node_trace = go.Scatter(
            x=node_x,
            y=node_y,
            mode="markers+text",
            hoverinfo="text",
            hovertext=hover_texts,
            text=node_labels,  # Permanent labels for nodes
            textposition="middle center",
            marker=dict(
                showscale=False,
                opacity=0.3,  # Decreased opacity of nodes for the desired transparency
                # colorscale='YlGnBu',
                size=20,
                colorbar=dict(
                    thickness=15,
                    title="Node Connections",
                    xanchor="left",
                    titleside="right",
                ),
                line_width=2,
                color=colors,
            ),
        )
        return node_trace

    def plot_network_on_timeline_interactive(
        self, savingpath=None, return_fig=True, adjust_overlap=True
    ):
        # Convert year attribute to integer, if not already
        # min_year, max_year, nodes_sorted_by_year = self.clean_years()

        self.pos = nx.kamada_kawai_layout(
            self.G,
            weight=None,
            scale=5.0,
            center=None,
            dim=2,
        )  # Use Kamada-Kawai layout for better node positioning

        if adjust_overlap:
            self.pos = self.adjust_overlap(0.15, 100)

        edge_trace = self.get_edge_trace()

        node_trace = self.get_node_trace()
        width = 1000
        height = 1500
        if len(self.G.nodes) > 190:
            width = 1300
            height = 1500
        # Figure Definition
        fig = go.Figure(
            data=[edge_trace, node_trace],
            layout=go.Layout(
                width=width,  # Example width
                height=height,  # Example height
                showlegend=False,  # Adjust based on your need to show or hide the legend
                hovermode="closest",
                hoverlabel=dict(  # Customizing the hover label's appearance
                    bgcolor="rgba(249, 98, 186, 0.11)",  # Setting the background color to a semi-transparent gray
                    bordercolor="rgba(0, 0, 0, 0.14)",
                    font=dict(color="black"),  # Text color in the hover label
                ),
                margin=dict(b=0, l=0, r=0, t=40),
                xaxis=dict(
                    showline=False,  # Hides the axis line
                    showgrid=False,
                    zeroline=False,
                    showticklabels=False,  # Hide tick labels
                    visible=False,  # Makes the whole axis (including the space for it) invisible
                ),
                yaxis=dict(
                    showline=False,  # Hides the axis line
                    showgrid=False,
                    zeroline=False,
                    showticklabels=False,  # Hide tick labels
                    visible=False,  # Makes the whole axis (including the space for it) invisible
                ),
                dragmode="zoom",
            ),
        )

        plt.tight_layout()
        if savingpath:
            if savingpath.endswith(".html"):  # Use 'savingpath' consistently
                fig.write_html(savingpath)
            elif savingpath.endswith(".png"):
                fig.write_image(savingpath, dpi=300)
        fig.show()
        if return_fig:
            return pio.to_html(fig, full_html=False, include_plotlyjs="cdn")


# example usage
# plotter = MainPathPlotterInteractive(graph, "cluster")
# plotter.plot_network_on_timeline_interactive(return_fig=False, adjust_overlap=True)
# plotter.plot_network_on_timeline_interactive(savingpath="network_timeline.html")
# plotter.plot_network_on_timeline_interactive(savingpath="network_timeline.png")

In [ ]:
plotter = MainPathPlotterInteractive(
    Gmp, "cluster_alpha0.3_k20_res0.006", "unique_auth_year"
)
fig = plotter.plot_network_on_timeline_interactive(return_fig=True, adjust_overlap=True)

<Figure size 640x480 with 0 Axes>

In [ ]:
clusters = len(
    set(nx.get_node_attributes(Gmp, "cluster_alpha0.3_k20_res0.006").values())
)

In [ ]:
clusters

36

In [ ]:
# mean year of cluster 5
df[df["cluster_alpha0.3_k20_res0.006"] == 4]["year"].mean()

2015.9114114114113

In [ ]:
# dataframe from Gmp
nodesmp = pd.DataFrame(**Gmp.nodes(data=True))

TypeError: pandas.core.frame.DataFrame() argument after ** must be a mapping, not NodeDataView

In [ ]:
# Convert the nodes and their attributes into a pandas DataFrame
nodesmp = pd.DataFrame.from_dict(dict(Gmp.nodes(data=True)), orient="index")

# If you want to include the node ID as a column as well, you can reset the index:
nodesmp.reset_index(inplace=True)
nodesmp.rename(columns={"index": "node_id"}, inplace=True)

nodesmp  # ["cluster_alpha0.3_k20_res0.006"].value_counts()

,node_id,label,x,y,size,shape,eid,unique_auth_year
0,1,"""family_2_62""",0.6974,0.9217,0.5000,ellipse,2-s2.0-0020047901;2-s2.0-0019989539,Boeck_1982;Pedersen_1982
1,2,"""47""",0.6737,0.9343,0.5000,ellipse,2-s2.0-0020069956,Baker_1982
2,3,"""family_33_36_37_40""",0.6782,0.9152,0.5000,ellipse,2-s2.0-0020425640;2-s2.0-0020416931;2-s2.0-002...,Overo_1982_2;Hyttel_1982;Ofsti_1982;Overo_1982
3,4,"""43""",0.6676,0.8984,0.5000,ellipse,2-s2.0-0020363953,Gastpar_1982
4,5,"""16""",0.2052,0.8995,0.5000,ellipse,2-s2.0-0019996341,Magnussen_1982
...,...,...,...,...,...,...,...,...
180,181,"""35944""",0.3830,0.0639,0.5000,ellipse,2-s2.0-85149225801,Venkatachalam_2023
181,182,"""36607""",0.4687,0.1007,0.5000,ellipse,2-s2.0-85161263006,Ferreira_2023
182,183,"""36624""",0.3755,0.0457,0.5000,ellipse,2-s2.0-85174155976,Fong_2023
183,184,"""36764""",0.4871,0.0906,0.5000,ellipse,2-s2.0-85177791552,Polverino_2023


In [ ]:
19. Key-Route Global Main Path [200-300] of N5 (237).net
22. Key-Route Local Main Path [100-200] [0.010] of N5 (213).net

10. Key-Route Global Main Path [200-250] of N5 (185).net

SyntaxError: invalid syntax (504417644.py, line 1)

In [9]:

# Get the list of nodes from the graph
all_nodes = list(G.nodes())

# Set a random seed for reproducibility
random.seed(42)

# Sample 1000 nodes randomly
sample_nodes = random.sample(all_nodes, 1000)

# Create a subgraph containing only the sampled nodes
G_sample = G.subgraph(sample_nodes)

In [16]:
# import graphml
import networkx as nx
G = nx.read_graphml("/Users/jlq293/Projects/Study-1-Bibliometrics/data/05-graphs/citation-graph/directed_citation_graph.graphml")

# keep a random sample of 1000 nodes
import random

# Get the list of nodes from the graph
all_nodes = list(G.nodes())

# Set a random seed for reproducibility
random.seed(42)

# Sample 1000 nodes randomly
sample_nodes = random.sample(all_nodes, 1000)

# Create a subgraph containing only the sampled nodes
G_sample = G.subgraph(sample_nodes)

# save the sample graph
nx.write_graphml(G_sample, "/Users/jlq293/Projects/Random Projects/LW-ThreeJS/SSRI/dat/directed_citation_graph_sample_1000.graphml")

# save a nodes df with x, y, z positions
pos = nx.fruchterman_reingold_layout(G_sample, dim=3)

# save to csv
df = pd.DataFrame(pos).T
df.columns = ["x", "y", "z"]
df.to_csv("/Users/jlq293/Projects/Random Projects/LW-ThreeJS/SSRI/dat/nodes_positions.csv")
print(df.head())

# edge list to df
edges = nx.to_pandas_edgelist(G_sample)

# add columns for source and target positions
edges["source_x"] = edges["source"].map(df["x"])
edges["source_y"] = edges["source"].map(df["y"])
edges["source_z"] = edges["source"].map(df["z"])
edges["target_x"] = edges["target"].map(df["x"])
edges["target_y"] = edges["target"].map(df["y"])
edges["target_z"] = edges["target"].map(df["z"])

edges.to_csv("/Users/jlq293/Projects/Random Projects/LW-ThreeJS/SSRI/dat/edges.csv", index=False)

print(edges.head())

                      x         y         z
Lan_2022       0.570199  0.197060 -0.848891
Hillner_2010   0.912882 -0.380962  0.199544
Abdrabou_2018 -0.932546  0.105258 -0.269369
Elmore_2005    0.482980  0.901701 -0.169172
Renshaw_2009   0.547643 -0.779110  0.395488
          source             target  source_x  source_y  source_z  target_x  \
0     Lynch_2011       Kennard_2006 -0.300813  0.469833  0.587177 -0.375139   
1    Wong_1985_3        Wong_1985_2 -0.758035  0.006398  0.022101 -0.996214   
2   Sanchez_2000  Przegalinski_1995 -0.363713 -0.581915 -0.029803 -0.539515   
3   Knorr_2012_3     Loubinoux_2005  0.636006 -0.330642  0.172113  0.860584   
4  Tzvetkov_2008        Peters_2004  0.446947 -0.476240  0.133634  0.546809   

   target_y  target_z  
0  0.584251  0.728445  
1  0.014303  0.032628  
2 -0.854955 -0.027199  
3 -0.436781  0.231669  
4 -0.595723  0.158523  
